روی داده های برق داده شده تمام کارهای مورد نیار را انجام داده و در انتها داده با فیچر های زیر را نیز پیش بینی کنید

"CO2Intensity"=491.32, "SystemLoadEA"=4241.05, "Day"=10, "ForecastWindProduction"=54.10, , "SMPEA"=49.56, "ORKTemperature"=9.0, "ORKWindspeed"=14.8, "ActualWindProduction"=54.0, "Month"=12, "SystemLoadEP2"=4426.84]

In [132]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.special import boxcox

In [133]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

from sklearn.linear_model import LinearRegression, Ridge, ElasticNetCV,SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn import model_selection
from sklearn.neighbors import KNeighborsRegressor


from sklearn.metrics import mean_squared_error,make_scorer,r2_score,confusion_matrix,mean_absolute_error
from sklearn import preprocessing
from sklearn import metrics
import math

In [134]:
data = pd.read_csv('electricity.csv',low_memory=False,na_values='?',index_col=1, parse_dates=True)

In [135]:
data.head(3)

,DateTime,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay,ForecastWindProduction,SystemLoadEA,SMPEA,ORKTemperature,ORKWindspeed,CO2Intensity,ActualWindProduction,SystemLoadEP2,SMPEP2
Holiday,,,,,,,,,,,,,,,,,
None,01/11/2011 00:00,0,1,44,1,11,2011,0,315.31,3388.77,49.26,6.0,9.3,600.71,356.0,3159.60,54.32
None,01/11/2011 00:30,0,1,44,1,11,2011,1,321.80,3196.66,49.26,6.0,11.1,605.42,317.0,2973.01,54.23
None,01/11/2011 01:00,0,1,44,1,11,2011,2,328.57,3060.71,49.10,5.0,11.1,589.97,311.0,2834.00,54.23


In [136]:
data.isna().sum()

DateTime                    0
HolidayFlag                 0
DayOfWeek                   0
WeekOfYear                  0
Day                         0
Month                       0
Year                        0
PeriodOfDay                 0
ForecastWindProduction      5
SystemLoadEA                2
SMPEA                       2
ORKTemperature            295
ORKWindspeed              299
CO2Intensity                7
ActualWindProduction        5
SystemLoadEP2               2
SMPEP2                      2
dtype: int64

In [137]:
data.describe()

,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay,ForecastWindProduction,SystemLoadEA,SMPEA,ORKTemperature,ORKWindspeed,CO2Intensity,ActualWindProduction,SystemLoadEP2,SMPEP2
count,38014.000000,38014.000000,38014.000000,38014.000000,38014.000000,38014.000000,38014.000000,38009.000000,38012.000000,38012.000000,37719.000000,37715.000000,38007.000000,38009.000000,38012.000000,38012.000000
mean,0.040406,2.997317,28.124586,15.739412,6.904246,2012.383859,23.501105,544.261451,4020.085019,62.720388,9.626369,19.211770,479.373040,520.762819,3785.973841,64.136823
std,0.196912,1.999959,15.587575,8.804247,3.573696,0.624956,13.853108,414.364629,860.476866,32.252334,4.439934,9.571311,85.354706,378.282975,843.269455,35.415036
min,0.000000,0.000000,1.000000,1.000000,1.000000,2011.000000,0.000000,0.680000,2183.940000,0.000000,-4.000000,0.000000,0.000000,1.000000,1809.960000,-47.740000
25%,0.000000,1.000000,15.000000,8.000000,4.000000,2012.000000,12.000000,189.670000,3281.207500,45.530000,6.000000,13.000000,421.105000,199.000000,3058.277500,45.780000
50%,0.000000,3.000000,29.000000,16.000000,7.000000,2012.000000,24.000000,441.980000,4103.600000,55.230000,9.000000,18.500000,480.310000,445.000000,3865.745000,55.545000
75%,0.000000,5.000000,43.000000,23.000000,10.000000,2013.000000,35.750000,839.460000,4638.532500,70.320000,13.000000,24.100000,537.520000,793.000000,4427.590000,72.110000
max,1.000000,6.000000,52.000000,31.000000,12.000000,2013.000000,47.000000,1680.000000,6492.910000,587.580000,25.000000,75.900000,842.880000,1769.000000,6309.750000,1000.000000


In [138]:
data = data.dropna(axis=0)

In [139]:
data.isna().sum()

DateTime                  0
HolidayFlag               0
DayOfWeek                 0
WeekOfYear                0
Day                       0
Month                     0
Year                      0
PeriodOfDay               0
ForecastWindProduction    0
SystemLoadEA              0
SMPEA                     0
ORKTemperature            0
ORKWindspeed              0
CO2Intensity              0
ActualWindProduction      0
SystemLoadEP2             0
SMPEP2                    0
dtype: int64

In [140]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37682 entries, None to New Year's Eve
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DateTime                37682 non-null  object 
 1   HolidayFlag             37682 non-null  int64  
 2   DayOfWeek               37682 non-null  int64  
 3   WeekOfYear              37682 non-null  int64  
 4   Day                     37682 non-null  int64  
 5   Month                   37682 non-null  int64  
 6   Year                    37682 non-null  int64  
 7   PeriodOfDay             37682 non-null  int64  
 8   ForecastWindProduction  37682 non-null  float64
 9   SystemLoadEA            37682 non-null  float64
 10  SMPEA                   37682 non-null  float64
 11  ORKTemperature          37682 non-null  float64
 12  ORKWindspeed            37682 non-null  float64
 13  CO2Intensity            37682 non-null  float64
 14  ActualWindProduction    37682 n

In [141]:
data.columns

Index(['DateTime', 'HolidayFlag', 'DayOfWeek', 'WeekOfYear', 'Day', 'Month',
       'Year', 'PeriodOfDay', 'ForecastWindProduction', 'SystemLoadEA',
       'SMPEA', 'ORKTemperature', 'ORKWindspeed', 'CO2Intensity',
       'ActualWindProduction', 'SystemLoadEP2', 'SMPEP2'],
      dtype='object')

In [145]:
data.HolidayFlag.value_counts()

0    36266
1     1416
Name: HolidayFlag, dtype: int64

In [146]:
data.DayOfWeek.value_counts()

1    5414
5    5402
4    5399
3    5380
0    5375
2    5362
6    5350
Name: DayOfWeek, dtype: int64

In [147]:
data.ActualWindProduction.value_counts()

54.0      75
46.0      73
58.0      71
53.0      71
48.0      70
          ..
1502.0     1
1501.0     1
1553.0     1
1516.0     1
1500.0     1
Name: ActualWindProduction, Length: 1535, dtype: int64

In [148]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 37682 entries, None to New Year's Eve
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   DateTime                37682 non-null  object 
 1   HolidayFlag             37682 non-null  int64  
 2   DayOfWeek               37682 non-null  int64  
 3   WeekOfYear              37682 non-null  int64  
 4   Day                     37682 non-null  int64  
 5   Month                   37682 non-null  int64  
 6   Year                    37682 non-null  int64  
 7   PeriodOfDay             37682 non-null  int64  
 8   ForecastWindProduction  37682 non-null  float64
 9   SystemLoadEA            37682 non-null  float64
 10  SMPEA                   37682 non-null  float64
 11  ORKTemperature          37682 non-null  float64
 12  ORKWindspeed            37682 non-null  float64
 13  CO2Intensity            37682 non-null  float64
 14  ActualWindProduction    37682 n

In [149]:
data.corr(numeric_only=True)

,HolidayFlag,DayOfWeek,WeekOfYear,Day,Month,Year,PeriodOfDay,ForecastWindProduction,SystemLoadEA,SMPEA,ORKTemperature,ORKWindspeed,CO2Intensity,ActualWindProduction,SystemLoadEP2,SMPEP2
HolidayFlag,1.000000,-0.132555,-0.025538,0.030917,0.012688,-0.010826,0.001025,0.073655,-0.053758,0.003094,-0.096742,0.062964,-0.054537,0.065255,-0.077773,-0.001838
DayOfWeek,-0.132555,1.000000,0.008881,-0.004845,0.004288,0.000978,0.000328,-0.035666,-0.184712,-0.077541,-0.015850,-0.053902,0.029910,-0.032427,-0.190931,-0.069625
WeekOfYear,-0.025538,0.008881,1.000000,0.056717,0.970971,-0.234029,-0.000263,0.044062,-0.027475,-0.017929,0.155844,-0.028194,-0.093605,0.062363,-0.039600,-0.015814
Day,0.030917,-0.004845,0.056717,1.000000,0.004558,0.001404,0.000059,0.070681,-0.007528,-0.017830,0.014871,0.086643,-0.082226,0.060629,-0.026060,-0.012801
Month,0.012688,0.004288,0.970971,0.004558,1.000000,-0.233672,-0.000189,0.047053,-0.032178,-0.017667,0.150300,-0.028385,-0.104434,0.064416,-0.041150,-0.014918
Year,-0.010826,0.000978,-0.234029,0.001404,-0.233672,1.000000,0.000176,-0.045914,-0.081615,0.045147,0.079474,-0.013096,-0.192302,-0.064141,-0.059444,0.045456
PeriodOfDay,0.001025,0.000328,-0.000263,0.000059,-0.000189,0.000176,1.000000,0.034860,0.609273,0.356120,0.137944,0.088345,-0.177251,0.061086,0.594772,0.323490
ForecastWindProduction,0.073655,-0.035666,0.044062,0.070681,0.047053,-0.045914,0.034860,1.000000,0.177769,-0.046090,-0.172912,0.714234,-0.652286,0.953812,0.034687,-0.079639
SystemLoadEA,-0.053758,-0.184712,-0.027475,-0.007528,-0.032178,-0.081615,0.609273,0.177769,1.000000,0.532210,-0.086961,0.182608,-0.209947,0.200531,0.972738,0.491096
SMPEA,0.003094,-0.077541,-0.017929,-0.017830,-0.017667,0.045147,0.356120,-0.046090,0.532210,1.000000,0.018898,0.014079,-0.086420,-0.032218,0.534822,0.618158


In [150]:
data.columns

Index(['DateTime', 'HolidayFlag', 'DayOfWeek', 'WeekOfYear', 'Day', 'Month',
       'Year', 'PeriodOfDay', 'ForecastWindProduction', 'SystemLoadEA',
       'SMPEA', 'ORKTemperature', 'ORKWindspeed', 'CO2Intensity',
       'ActualWindProduction', 'SystemLoadEP2', 'SMPEP2'],
      dtype='object')

In [161]:
X = data.drop(['DateTime','SMPEP2'],axis=1)
y = data[['SMPEP2']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 0)

In [162]:
models = [LinearRegression(),Ridge(),DecisionTreeRegressor(), RandomForestRegressor(),ExtraTreesRegressor(),
         SGDRegressor(),GradientBoostingRegressor(),KNeighborsRegressor()]

In [163]:
def re(models):
    for i in models:   
        md = i
        
        md.fit(X_train, y_train)
        md.fit(X_train, y_train)
        y_predict_md= md.predict(X_test)
        
               
        print("*****************",i,"*****************")
        print("MSE : ", metrics.mean_squared_error(y_test, y_predict_md))
        print("R2_SCORE : ", metrics.r2_score(y_test, y_predict_md))
        print("MAPE : ", metrics.mean_absolute_error(y_test, y_predict_md,multioutput='uniform_average'))
        print("\n")

In [ ]:
re(models)

***************** LinearRegression() *****************
MSE :  679.9651970159889
R2_SCORE :  0.45076155388805206
MAPE :  14.491055272765944


***************** Ridge() *****************
MSE :  679.9632082251488
R2_SCORE :  0.4507631603237865
MAPE :  14.490981140850144


***************** DecisionTreeRegressor() *****************
MSE :  828.5814012471805
R2_SCORE :  0.3307175671704845
MAPE :  10.453474857370308




/tmp/ipykernel_9412/1625276923.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  md.fit(X_train, y_train)
/tmp/ipykernel_9412/1625276923.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  md.fit(X_train, y_train)


به صورت واترمارک کنید و خروجی را ذخیره کنید logo.pngروی عکس یکی از دوستان یا خانواده